## hyperband tuner

This is (hopefully) the final model tuner in this project. It should produce models with an acceptable accuracy for production.
Herein some hyper-parameters are adjusted from knowledge gained from the random tuner (e.g. `AdaDelta` is removed because it seemed to not perform well against the other optimizers).

The `RandomTuner` got replaced by `Hyperband` and a factor of 3 was given to the class (as suggested in the respective [paper](http://jmlr.org/papers/v18/16-558.html)).
`max_epochs` maximizes the number of the competing models, but this is an upper bound, since the most models will stop training due to `EarlyStopping`.

In [1]:
from os import path

import tensorflow as tf

from src.utils import decode_image_record

processed = path.join('data', 'processed')

features = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([2], tf.int64)
}
shape = (32, 32, 1)

def decoder(example):
    feature = tf.io.parse_single_example(example, features)
    image = tf.io.parse_tensor(feature['image'], tf.float32)
    image.set_shape(shape)
    # We only want the 'label_idx'. Not the 'angle'.
    label = feature['label'][0]

    return [image, label]


train_dataset = decode_image_record(path.join(processed, 'train.tfrecord'), decoder)
validation_dataset = decode_image_record(path.join(processed, 'validate.tfrecord'), decoder)
test_dataset = decode_image_record(path.join(processed, 'test.tfrecord'), decoder)

In [2]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop, SGD

def create_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(2**hp.Int('2**num_filter_0', 4, 6),
        (4,4) ,activation='relu', input_shape=(32, 32, 1)))

    for i in range(hp.Int('num_cnn_layers', 0, 3)):
        filter = 2**hp.Int('2**num_filter_' + str(i), 4, 7)
        model.add(layers.Conv2D(filter, (4,4), activation='relu', padding='same'))
        if hp.Boolean('pooling_' + str(i)):
            model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        nodes = 2**hp.Int('2**num_nodes_' + str(i), 4, 7)
        model.add(layers.Dense(nodes, activation='relu'))
    
    model.add(layers.Dense(3, 'softmax'))

    optimizers = {
        'adam': Adam(),
        'sgd': SGD(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]),
            momentum=hp.Float('momentum', 0.6, 1, 0.1),
            nesterov=hp.Boolean('nesterov')),
        'rms': RMSprop(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]))
    }

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizers[hp.Choice('optimizer', list(optimizers.keys()))],
        metrics=['acc'])

    return model

In [3]:
from datetime import datetime
from tensorboard.plugins.hparams import api
from kerastuner import Hyperband
from tensorflow import summary
from tensorflow.keras.callbacks import TensorBoard

class customTuner(Hyperband):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def on_trial_end(self, trial):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the hparams where the metrics of tensorboard are
        hparam_dir = path.join(trial_dir, trial.trial_id)
        hparams = trial.hyperparameters.values
        with summary.create_file_writer(hparam_dir).as_default():
            api.hparams(hparams, trial_id=trial.trial_id)

        print(datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
        
        super().on_trial_end(trial)

    def on_epoch_end(self, trial, model, epoch, logs):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the data where the metrics of tensorboard are
        hist_dir = path.join(trial_dir, trial.trial_id)
        with summary.create_file_writer(hist_dir).as_default():
            for layer in model.weights:
                summary.histogram(layer.name, data=layer, step=epoch)
        super().on_epoch_end(trial, model, epoch, logs)

In [4]:
from kerastuner import HyperParameters

hp=HyperParameters()
log_dir = path.join('logs', 'srp19')
timestamp = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

tuner = customTuner(
    create_model,
    hyperparameters=hp,
    objective='acc',
    max_epochs=200,
    factor=3,
    executions_per_trial=1,
    directory=log_dir,
    project_name=timestamp)

tuner.search_space_summary()

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [ EarlyStopping(monitor='loss', patience=3) ]

tuner.search(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    steps_per_epoch=100,
    validation_steps=100,
    verbose=0,
    callbacks=callbacks)

2019-11-21T18-59-49
2019-11-21T18-59-54
2019-11-21T18-59-58
...


In [6]:
tuner.results_summary()

Here the best results are examined. After evaluating the data in the logs and handpicking the best models some have gotten an accuracy of 96,7% on the test data.

Further training and manually handpicking the best hyper-parameters is done in designated python scripts using the same techniques as in these notebooks.

In [7]:
test_winner = tuner.get_best_models(num_models=10)

for winner in test_winner:
    print(winner.evaluate(test_dataset, steps=100, verbose=0))

[0.40828269101213666, 0.936875]
[0.31918077359441666, 0.9234375]
[0.4869032717659138, 0.9359375]
[0.38010222040116787, 0.909375]
[0.2098659366928041, 0.9434375]
[0.23279909185133874, 0.9440625]
[0.416561471009627, 0.915625]
[0.19408582380914596, 0.93875]
[0.21677776782773436, 0.92]
[0.21536825909046456, 0.9465625]
